## 그라디오의 챗봇에 연동하기

### 0-1. api_key 불러오기

In [18]:
from dotenv import load_dotenv
import os

load_dotenv()

True

### 0-2. Assistant 응답 결과를 json 형태로 반환하는 함수 만들기

In [19]:
import json

def response_to_dict(obj):
    return json.loads(obj.model_dump_json())

In [20]:
from openai import OpenAI
client = OpenAI()

my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
for i in my_assistants.data:
    print(i)

Assistant(id='asst_GK2DimTvpiFrWBDbCOq9DwjQ', created_at=1712988472, description=None, file_ids=['file-DqdIDulmB8mP3GCjAk3NGTEX', 'file-HyeZozp2SL4fjaR2EZDPvVZq', 'file-4GMJdPNpAJvR8vJ1zv0sgjBI', 'file-lgVC1S54zKZOUkeGLtuJZX4A', 'file-CoVYbf5ZNpYnYEML2FoPMdqn', 'file-qPfDpph7Ouhi4Y51K4sObBgP', 'file-rTNSIUTUjG5l1yikWILW9pa1'], instructions="This GPT is designed to answer questions based on the contents of a specific text related to a game, focusing on game-related topics, especially those concerning characters, plot points, strategies, and any in-game mechanics detailed within the document. It will use the provided text as a reference to provide accurate and detailed responses. When asked a question, it will either provide information directly from the document or infer answers based on the content when direct information is not available. The GPT will clarify or ask for more details if the question is too broad or if it cannot find the exact information in the document. It aims to eng

### 1-1. 만들어 놓은 assistant 에 연결

In [21]:
my_assistant = client.beta.assistants.retrieve(
    assistant_id="asst_GK2DimTvpiFrWBDbCOq9DwjQ"
)
print(my_assistant)

Assistant(id='asst_GK2DimTvpiFrWBDbCOq9DwjQ', created_at=1712988472, description=None, file_ids=['file-DqdIDulmB8mP3GCjAk3NGTEX', 'file-HyeZozp2SL4fjaR2EZDPvVZq', 'file-4GMJdPNpAJvR8vJ1zv0sgjBI', 'file-lgVC1S54zKZOUkeGLtuJZX4A', 'file-CoVYbf5ZNpYnYEML2FoPMdqn', 'file-qPfDpph7Ouhi4Y51K4sObBgP', 'file-rTNSIUTUjG5l1yikWILW9pa1'], instructions="This GPT is designed to answer questions based on the contents of a specific text related to a game, focusing on game-related topics, especially those concerning characters, plot points, strategies, and any in-game mechanics detailed within the document. It will use the provided text as a reference to provide accurate and detailed responses. When asked a question, it will either provide information directly from the document or infer answers based on the content when direct information is not available. The GPT will clarify or ask for more details if the question is too broad or if it cannot find the exact information in the document. It aims to eng

### 2-1. 새 대화창 생성

In [ ]:
my_thread = client.beta.threads.create() # 스레드 생성 : 새 대화창이라고 생각

### 3-1. 그라디오 챗봇 만들기

In [12]:
# !pip install -qq gradio==4.10

In [23]:
import gradio as gr

In [24]:
def game_guide(message, chat_history):
    if not message:
        return "", chat_history
    else :
        thread_message = client.beta.threads.messages.create(
        thread_id=my_thread.id,
        role="user",
        content=message,
        )

        run = client.beta.threads.runs.create_and_poll(
        thread_id=my_thread.id,
        assistant_id="asst_GK2DimTvpiFrWBDbCOq9DwjQ"
        )
        
        if run.status == 'completed': 
            messages = client.beta.threads.messages.list(
                thread_id=my_thread.id
            )
            response_data = response_to_dict(messages)
            message_data = response_data["data"][0]
            response_message = message_data["content"][0]["text"]["value"]
        else:
            print(run.status)
    chat_history.append([message, response_message])
    return "", chat_history

In [25]:
with gr.Blocks(theme=gr.themes.Default()) as app:
    with gr.Tab("데이브 더 다이브 게임 가이드"):
        gr.Markdown(
            value="<center>데이브 더 다이브 게임 가이드입니다</center>"
            )
        with gr.Row():
            cb_chatbot = gr.Chatbot(
                value=[[None, "안녕하세요 데이브 더 다이브 게임 가이드 입니다"]],
                show_label=False
            )
        with gr.Row():
            cb_user_input = gr.Text(
                lines=1,
                placeholder="입력창",
                container=False,
                scale=9
            )
            cb_send_btn = gr.Button(
                value="보내기",
                scale=1,
                variant="primary"
            )
            cb_send_btn.click(
                fn=game_guide,
                inputs=[cb_user_input, cb_chatbot],
                outputs=[cb_user_input, cb_chatbot]
            )
            cb_user_input.submit(
                                fn=game_guide,
                inputs=[cb_user_input, cb_chatbot],
                outputs=[cb_user_input, cb_chatbot]
            )
app.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
